# Transformaciones<a class="tocSkip">
## TRATAMIENTO DE SEÑALES <a class="tocSkip">
### Ingenieria Electrónica <a class="tocSkip">
### Universidad Popular del Cesar <a class="tocSkip">
### Prof.: Jose Ramón Iglesias Gamarra - [https://github.com/joseramoniglesias/](https://github.com/joseramoniglesias/) <a class="tocSkip">
  **joseiglesias@unicesar.edu.co**

# Transformaciones de Imagen - DIP
Hasta ahora hemos discutido operaciones de procesamiento que trabajan directamente sobre los píxeles de una imagen de entrada (*dominio espacial*). En algunos casos, las operaciones de procesamiento se benefician de aplicar alguna transformación a la imagen de entrada, aplicar la operación en el nuevo dominio y aplicar la transformación inversa para volver al dominio espacial.

<img src="https://github.com/MoraRubio/dip-uam/blob/main/src/transformaciones-imagen.png?raw=true" alt="Operaciones en el dominio de la transformada" width="1000"/>

Un conjunto importante de estas transformaciones se puede expresar de la forma:

$$T(u,v)=\sum_{x=0}^{M-1}\sum_{y=0}^{N-1}f(x,y)r(x,y,u,v)$$

donde $r(x,y,u,v)$ se denomina *kernel de transformación*.

Y la correspondiente transformación inversa:

$$f(x,y)=\sum_{u=0}^{M-1}\sum_{v=0}^{N-1}T(u,v)s(x,y,u,v)$$

donde $s(x,y,u,v)$ se denomina *kernel de transformación inversa*.

Una transformada de particular interés en el procesamiento digital de imágenes es la transformada de Fourier, cuyos kernels definen como:

$$r(x,y,u,v)=e^{-j2\pi (ux/M+vy/M)}$$

$$s(x,y,u,v)=\frac{1}{MN}e^{j2\pi (ux/M+vy/M)}$$

Las transformadas *Walsh*, *Hadamard*, *discrete cosine*, *Haar*, y *slant* también se pueden expresar de esta forma.

## Acerca de la transformada de Fourier
Recibe su nombre por el matemático francés Jean Baptiste Joseph Fourier, quien propuso alrededor del año 1820 qué:

Cualquier señal periódica puede ser representada como una suma de funciones seno o coseno de diferentes frecuencias, cada una multiplicada por un coeficiente diferente (*Serie de Fourier*)

<img src="https://github.com/MoraRubio/dip-uam/blob/main/src/fourier.png?raw=true" alt="Ejemplo Fourier libro" width="350"/>

Las funciones no periódicas, pero cuya área bajo la curva es finita, se pueden expresar como la integral de senos o cosenos multiplicados por una función de coeficientes (*Transformada de Fourier*).

<img src="https://upload.wikimedia.org/wikipedia/commons/7/72/Fourier_transform_time_and_frequency_domains_%28small%29.gif" alt="Ejemplo Fourier Wikipedia" width="400"/>

Una característica importante, tanto de la serie como de la transformada de Fourier, es que la función representada puede ser reconstruida o recuperada por completo sin pérdida de información, lo que permite trabajar en el dominio de Fourier (*dominio de la frecuencia*) y luego volver al dominio original.

Desde el punto de vista computacional, actualmente se usa una implementación llamada transformada rápida de Fourier (FFT, Fast Fourier Transform) la cual permite calcular de manera rápida la transformada discreta incluso para muchos datos.

El ejemplo más común es el de filtrado en el dominio de la frecuencia, el cual es más eficiente que el filtrado espacial visto anteriormente.

<img src="https://github.com/MoraRubio/dip-uam/blob/main/src/complejidadFFT.png?raw=true" alt="Complejidad filtrado en frecuencia vs espacial" width="500"/>

## Configuración del ambiente

In [ ]:
# Importamos las librerías necesarias
import cv2
import numpy as np
from scipy import signal
import scipy.fftpack as fp
from skimage.draw import disk
import matplotlib.pyplot as plt

%matplotlib inline

def signaltonoise(a, axis=0, ddof=0):
    a = np.asanyarray(a)
    m = a.mean(axis)
    sd = a.std(axis = axis, ddof = ddof)
    return np.where(sd == 0, 0, m / sd)

def plot_images(img, title=None, font_size=None, axis="off", color=cv2.COLOR_BGR2RGB):
    n_imgs = len(img)
    if  n_imgs > 1:
        _, axs = plt.subplots(1, n_imgs, **{'figsize':(3*n_imgs, 3)})
        axs = axs.ravel()
        for i in range(n_imgs):
            if title and (len(title) == n_imgs):
                axs[i].set_title(title[i], fontsize=font_size)
            axs[i].axis(axis)
            if len(img[i].shape) == 2:
              axs[i].imshow(img[i], cmap="gray")
            else:
              axs[i].imshow(cv2.cvtColor(img[i], color))
        plt.tight_layout()    
    else:
        plt.title(title, fontsize=font_size)
        plt.axis(axis)
        if len(img[0].shape) == 2:
          plt.imshow(img[0], cmap="gray")
        else:
          plt.imshow(cv2.cvtColor(img[0], color))

In [ ]:
# Leemos las imágenes y nos aseguramos de que sean binarias
lenna = cv2.resize(cv2.imread("lenna.jpg", cv2.IMREAD_GRAYSCALE), (512, 512))
astronauta = cv2.resize(cv2.imread("astronauta.png", cv2.IMREAD_GRAYSCALE), (512, 512))
pcb = cv2.resize(cv2.imread("pcb.png", cv2.IMREAD_GRAYSCALE), (512, 512))
plot_images([lenna, astronauta, pcb])

## Cálculo y visualización de la FFT

In [ ]:
freq = fp.fft2(lenna) # Para el cálculo de la FFT usamos la librería SciPy
imageOut = fp.ifft2(freq).real # Cálculo de la FFT inversa

assert(np.allclose(lenna, imageOut)) # Verificación de que la imagen original (lenna) y la reconstruida (imageOut) tengan valores cercanos

plot_images([lenna, imageOut.astype(int)], ["Original", "Reconstruida"])

In [ ]:
freq_dB = 20*np.log10(0.01+freq) # Paso de escala lineal a escala logarítmica para mejor visualización
plot_images([freq_dB.real.astype(int)]) # Seleccionamos la parte real y convertimos a número entero de 8 bits

In [ ]:
freq2 = fp.fftshift(freq) # Centramos el espectro para una mejor visualización
freq_dB = 20*np.log10(0.01+freq2) # Paso de escala lineal a escala logarítmica
plot_images([freq_dB.real.astype(int)]) # Seleccionamos la parte real y convertimos a número entero de 8 bits

In [ ]:
spectrum = 20*np.log10(0.01+np.abs(fp.fftshift(freq)))
phase = np.angle(fp.fftshift(freq))

plot_images([lenna, spectrum.real.astype(int), phase.real.astype(int), imageOut.astype(int)], 
            ["Original", "Espectro FFT", "Fase FFT", "Reconstruida"])

In [7]:
freq_pcb = fp.fft2(pcb) # Para el cálculo de la FFT usamos la librería ScipPy
imageOut = fp.ifft2(freq_pcb).real # Cálculo de la FFT inversa
spectrum = 20*np.log10(0.01+np.abs(fp.fftshift(freq_pcb)))
phase = np.angle(fp.fftshift(freq_pcb))
plot_images([pcb, spectrum.real.astype(int), phase.real.astype(int), imageOut.astype(int)], 
            ["Original", "Espectro FFT", "Fase FFT", "Reconstruida"])

In [ ]:
im1_ = fp.ifft2(np.vectorize(complex)(freq.real, freq_pcb.imag)).real
im2_ = fp.ifft2(np.vectorize(complex)(freq_pcb.real, freq.imag)).real
plot_images([np.clip(im2_, 0, 255).astype(int), np.clip(im2_, 0, 255).astype(int)], ["Reconstruida [Re(lenna)+Im(pcb)]", "[Re(pcb)+Im(lenna)]"])

## Filtrado en frecuencia

De acuerdo al teorema de la convolución, en el dominio de la frecuencia realizamos una multiplicación entre el espectro de nuestra imagen y el espectro del filtro.

### Filtro pasabajas

In [ ]:
shape = lenna.shape # Tamaño de la imagen
center_X, center_Y = shape[0]//2, shape[1]//2 # Centro de la imagen
r = 10 # Radio del filtro, a mayor tamaño, mayor efecto de suavizado
rr, cc = disk((center_X, center_Y), r) # Coordenadas del círculo de radio r centrado en la imagen
kernel = np.zeros(shape, dtype=np.uint8)
kernel[rr, cc] = 1 # Creación del filtro

freq = fp.fft2(lenna)

freq_kernel = fp.fft2(fp.ifftshift(kernel))
convolved = freq*freq_kernel
imageOut = fp.ifft2(convolved).real

lenna_spectrum = (20*np.log10(0.01+fp.fftshift(freq))).real.astype(int)
kernel_spectrum = (20*np.log10(0.01+fp.fftshift(freq_kernel))).real.astype(int)
result_spectrum = (20*np.log10(0.01+fp.fftshift(convolved))).real.astype(int)

plot_images([lenna, 255*kernel, imageOut], ["Original", "Filtro pasabajas", "Resultado"])
plot_images([lenna_spectrum, kernel_spectrum, result_spectrum], ["Espectro FFT Original", "Espectro FFT Filtro pasabajas", "Espectro FFT Resultado"])

Con un kernel gaussiano

In [ ]:
std = 10 # Dispersión de la distribución gaussiana, a mayor dispersión, mayor efecto de suavizado
gauss_kernel = np.outer(signal.gaussian(lenna.shape[0], std), signal.gaussian(lenna.shape[1], std))

freq = fp.fft2(lenna)
freq_kernel = fp.fft2(fp.ifftshift(gauss_kernel))
convolved = freq*freq_kernel
imageOut = fp.ifft2(convolved).real

lenna_spectrum = (20*np.log10(0.01+fp.fftshift(freq))).real.astype(int)
kernel_spectrum = (20*np.log10(0.01+fp.fftshift(freq_kernel))).real.astype(int)
result_spectrum = (20*np.log10(0.01+fp.fftshift(convolved))).real.astype(int)

plot_images([lenna, 255*gauss_kernel, imageOut], ["Original", "Kernel Gaussiano", "Resultado"])
plot_images([lenna_spectrum, kernel_spectrum, result_spectrum], ["Espectro FFT Original", "Espectro FFT Kernel Gaussiano", "Espectro FFT Resultado"])

### Comparando el filtrado espacial vs el filtrado en frecuencia

In [ ]:
gauss_kernel = np.outer(signal.gaussian(lenna.shape[0], 10), signal.gaussian(lenna.shape[1], 10))
freq = fp.fft2(lenna)

freq_kernel = fp.fft2(fp.ifftshift(gauss_kernel))
convolved = freq*freq_kernel
imageOut = fp.ifft2(convolved).real

filtrado_espacial = cv2.GaussianBlur(lenna, (25,25), 20.0)
freq_filtrado = fp.fft2(filtrado_espacial)

lenna_spectrum = (20*np.log10(0.1+fp.fftshift(freq))).real.astype(int)
kernel_spectrum = (20*np.log10(0.1+fp.fftshift(freq_filtrado))).real.astype(int)
result_spectrum = (20*np.log10(0.1+fp.fftshift(convolved))).real.astype(int)

plot_images([lenna, filtrado_espacial, imageOut], ["Original", "Filtro Gaussiano espacial", "Filtro Gaussiano en frecuencia"])
plot_images([lenna_spectrum, kernel_spectrum, result_spectrum], ["Espectro FFT Original", "Espectro FFT Filtro Gaussiano espacial", "Espectro FFT Filtro Gaussiano en frecuencia"])

Otra opción es modificar directamente el espectro de la imagen utilizando máscaras binarias

In [ ]:
# Pasaaltas
shape = lenna.shape
center_X, center_Y = shape[0]//2, shape[1]//2
rr, cc = disk((center_X, center_Y), 30)
kernel = np.ones(shape, dtype=int)
kernel[rr, cc] = 0

freq = fp.fftshift(fp.fft2(lenna))

freq_kernel = fp.fft2(fp.ifftshift(kernel))
convolved = freq*kernel
imageOut = np.clip(fp.ifft2(fp.ifftshift(convolved)).real,0,255)

lenna_spectrum = (20*np.log10(0.1+freq)).real.astype(int)
result_spectrum = (20*np.log10(0.1+convolved)).real.astype(int)

plot_images([lenna, 255*kernel, imageOut], ["Original", "Filtro pasaaltas", "Resultado"])
plot_images([lenna_spectrum, result_spectrum], ["Espectro FFT Original", "Espectro FFT Resultado"])

In [ ]:
shape = astronauta.shape
center_X, center_Y = shape[0]//2, shape[1]//2
rr, cc = disk((center_X, center_Y), 50)
kernel = np.zeros(shape, dtype=np.uint8)
kernel[rr, cc] = 1

freq = fp.fftshift(fp.fft2(astronauta))

freq_kernel = fp.fft2(fp.ifftshift(kernel))
convolved = freq*kernel
imageOut = np.clip(fp.ifft2(fp.ifftshift(convolved)).real,0,255)

astronauta_spectrum = (20*np.log10(0.1+freq)).real.astype(int)
result_spectrum = (20*np.log10(0.1+convolved)).real.astype(int)

plot_images([astronauta, 255*kernel, imageOut], ["Original", "Filtro pasabajas", "Resultado"])
plot_images([astronauta_spectrum, result_spectrum], ["Espectro FFT Original", "Espectro FFT Resultado"])

### Filtros pasabandas

In [ ]:
shape = astronauta.shape
center_X, center_Y = shape[0]//2, shape[1]//2
rr, cc = disk((center_X, center_Y), 30)
kernel2 = np.zeros(shape, dtype=np.uint8)
kernel2[rr, cc] = 1

kernel3 = kernel-kernel2

freq = fp.fftshift(fp.fft2(astronauta))

freq_kernel = fp.fft2(fp.ifftshift(kernel3))
convolved = freq*kernel3
imageOut = np.clip(fp.ifft2(fp.ifftshift(convolved)).real,0,255)

astronauta_spectrum = (20*np.log10(0.1+freq)).real.astype(int)
result_spectrum = (20*np.log10(0.1+convolved)).real.astype(int)

plot_images([astronauta, 255*kernel3, imageOut], ["Original", "Filtro pasabajas", "Resultado"])
plot_images([astronauta_spectrum, result_spectrum], ["Espectro FFT Original", "Espectro FFT Resultado"])

### Filtro rechaza bandas

In [ ]:
shape = astronauta.shape
center_X, center_Y = shape[0]//2, shape[1]//2
rr, cc = disk((center_X, center_Y), 60)
kernel = np.zeros(shape, dtype=np.uint8)
kernel[rr, cc] = 1

rr, cc = disk((center_X, center_Y), 30)
kernel2 = np.zeros(shape, dtype=np.uint8)
kernel2[rr, cc] = 1

kernel3 = 1-(kernel-kernel2)

freq = fp.fftshift(fp.fft2(astronauta))

freq_kernel = fp.fft2(fp.ifftshift(kernel3))
convolved = freq*kernel3
imageOut = np.clip(fp.ifft2(fp.ifftshift(convolved)).real,0,255)

astronauta_spectrum = (20*np.log10(0.1+freq)).real.astype(int)
result_spectrum = (20*np.log10(0.1+convolved)).real.astype(int)

plot_images([astronauta, 255*kernel3, imageOut], ["Original", "Filtro pasabajas", "Resultado"])
plot_images([astronauta_spectrum, result_spectrum], ["Espectro FFT Original", "Espectro FFT Resultado"])

### Creando una máscar específica para eliminar la interferencia

In [ ]:
rr, cc = disk((center_X, center_Y), 30)
kernel = np.ones(shape, dtype=np.uint8)
kernel[rr, cc] = 0

freq = fp.fftshift(fp.fft2(astronauta))
astronauta_spectrum = (20*np.log10(0.1+freq)).real.astype(int)
mask = 1-(astronauta_spectrum>100)*kernel # Sin tener en cuenta el centro de la imagen, buscar los picos del espectro mayores a 100 que corresponden al ruido

freq_kernel = fp.fft2(fp.ifftshift(kernel))
convolved = freq*mask
imageOut = np.clip(fp.ifft2(fp.ifftshift(convolved)).real,0,255)

result_spectrum = (20*np.log10(0.1+convolved)).real.astype(int)

plot_images([astronauta, 255*mask, imageOut], ["Original", "Máscara", "Resultado"])
plot_images([astronauta_spectrum, result_spectrum], ["Espectro FFT Original", "Espectro FFT Resultado"])

## Ejercicio

Aplicar a una imagen ruido gaussiano y ruido sal y pimienta, determinar un filtro adecuado en frecuencia para reducir el ruido, aplicar el filtro y mostrar los resultados.

**Copyright**

The notebooks are provided as [Open Educational Resource](https://de.wikipedia.org/wiki/Open_Educational_Resources). Feel free to use the notebooks for your own educational purposes. The text is licensed under [Creative Commons Attribution 4.0](https://creativecommons.org/licenses/by/4.0/), the code of the IPython examples under the [MIT license](https://opensource.org/licenses/MIT).